## Part 1. Gaussian quadrature

The Gaussian quadrature method is one of the methods of numerical integration that allows you to increase the accuracy of integration, by using orthogonal polynomials and their roots as nodes.

$$
\int_a^b \!  w(x)\, f(x)\, dx \approx \sum_{k=1}^n w_k \, f(x_k) \;,
$$

here $\omega$ is the weight function, this weight function determines the basis of orthogonal polynomials whose roots are used as integration nodes. If the function $f(x)$ is a polynomial, then such an integration method gives *exact* value for the integral.

For example, let's calculate the following integral:
$$
\int_{-1}^1 (7x^3 - 8 x^2 - 3 x + 3) dx  \label{eq1}\tag{1}
$$

Here function is already polynomial, so we put the weight function $\omega(x) = 1$. Then the calculation of the integral reduces to the sum in the polynomials roots witch corresponding to the weight function $\omega(x) = 1$. For example, here https://dlmf.nist.gov/18.3 you may see which basis of polynomials corresponds to your weight function.

You may see, that Legendre polynomials correspond to our weight function. Now go to the documentation https://docs.scipy.org/doc/scipy/reference/special.html#orthogonal-polynomials. Here, the function *roots_legendre* immediately gives you the roots of the Legendre polynomial and weight at this roots (for Legendre polynomials all these weight are ones).


In [0]:
import numpy as np
import scipy
from scipy import special
from scipy.special import roots_legendre
from scipy.special import roots_chebyt

### Task 1.1
(20% of the grade)

1. Calculate the exact value of the integral ([1](#mjx-eqn-eq1)) using a pen and paper (or in your mind). Remember it, then you will compare numerical results with it.

In [3]:
print(2/3)

0.6666666666666666


2. Calculate the value of the integral ([1](#mjx-eqn-eq1)) using the usual method of numerical integration on a uniform grid (for example, implemented by you in week_6_calculus homework). Estimate the number of nodes needed to achieve relative accuracy $10^{-10}$.

In [0]:
def midpoint_rule(func, a, b, eps):
    answer = np.inf
    integral = (b - a) * func((b - a) / 2)
    n = 1
    while abs(integral - answer) > eps:
        answer = integral
        n *= 2
        d = (b - a) / n
        x = np.linspace(a + 0.5*d, b - 0.5*d, n)
        
        integral = sum([func(i) * d for i in x])
    return integral, n

In [12]:
func = lambda x: 7*x**3 - 8*x**2 - 3*x + 3
integral, n = midpoint_rule(func, -1, 1, 1e-10)
print(integral, n+1) 

0.6666666666860301 524289


3. Calculate the integral ([1](#mjx-eqn-eq1)) using the roots of the Legendre polynomial from 1 to 6 degrees as nodes (to get roots and weigths use scipy.special.roots_legendre)

In [0]:
def legendre_nodes_method(coef, n):
    x_n, w_n = special.roots_legendre(n) #n - степень полинома Лежандра
    return np.sum(w_n * np.polyval(coef, x_n))


In [24]:
coef=[7, -8, -3, 3]

for n in range(1,7):
    print('for n = ', n)
    print('integral = ', legendre_nodes_method(coef, n))

for n =  1
integral =  6.0
for n =  2
integral =  0.6666666666666665
for n =  3
integral =  0.6666666666666634
for n =  4
integral =  0.6666666666666667
for n =  5
integral =  0.6666666666666683
for n =  6
integral =  0.6666666666666696


Compare the number of nodes needed to obtain the same accuracy in both numerical methods.

Видно, что в данном случае в методе с использованием корней полинома Лежандра уже начиная со второго узла достаточно высокая точность, хотя в первом случае понадобилось больше 500 тысяч

### Task 1.2
(20% of the grade)

Calculate the value of the same integral, but on the interval from 0 to 10.
$$
\int_{0}^{10} (7x^3 - 8 x^2 - 3 x + 3) dx 
$$

Please note that you can no longer directly use Legendre polynomials, because they are defined for the interval (-1,1). But you can always make an *affine transformation* (a transformation that does not change the area) in order to go to the desired coordinate system.

Сделаем замену y=10x, тогда интеграл примет вид: 
$$
\int_{0}^{1} (7000 y^3 - 800 y^2 - 30 y + 3) dy 
$$

In [0]:
def legendre_nodes_sh_method(coef, n):
    x_n, w_n = special.roots_sh_legendre(n)
    return np.sum(w_n * np.polyval(coef, x_n))

In [32]:
coef_1=[70000,-8000,-300,30]

for n in range(1,7):
    print('for n = ', n)
    print('integral = ', legendre_nodes_sh_method(coef_1, n))

for n =  1
integral =  6630.0
for n =  2
integral =  14713.333333333334
for n =  3
integral =  14713.333333333336
for n =  4
integral =  14713.33333333333
for n =  5
integral =  14713.333333333332
for n =  6
integral =  14713.333333333332


Для 2 узла отличие между 14713.(3) и 14713.333333333334 порядка 10^(-12) -- высокая точность


### Task 1.3
(20% of the grade)

Calculate the value of the following integral:
$$
\int_{0}^{2\pi} \frac{cos(x) dx }{\sqrt{4\pi^2 - x^2}}
$$
by using the corresponding basis of orthogonal polynomials. 
Find the degree of the polynomial (number of nodes) at which the accuracy of the integral starts to exceed the double floating point accuracy.

Сделаем замену $$y={2\pi}x
$$
Тогда наш интеграл сведется к: $$
\int_{0}^{1} \frac{cos(2\pi y) dy }{\sqrt{1 - y^2}}
 = \int_{-1}^{1} \frac{cos(2\pi y) dy }{2\sqrt{1 - y^2}}
$$

In [0]:
def cheb_integral(n):
    x_n, w_n = special.roots_chebyt(n)
    return np.sum(w_n * np.cos(2*np.pi*x_n) / 2)

In [51]:
answer = np.inf
n = 1

while 0 == 0:
    
    integral = cheb_integral(n)
    if answer - integral == 0:
        break
    answer = integral
    n +=1
    
print(integral, n-1)

0.34601015881226443 16


Получили оптимальную степень полинома и приближенное значение интеграла

## Part 2. Fredholm equation

There are two types of Fredholm equations. 

1. Fredholm equation of the first kind:
$$
\int_a^b \! K(x, t)\, \color{blue}{f(t)} \,dt = g(x)
$$

2. Fredholm equation of the second kind:
$$
\lambda \int_a^b\! K(x, t)\, \color{blue}{f(t)} \, dt + g(x) = \color{blue}{f(x)}
$$

Here higtlighted function $f(x)$ is unknown, kernel function $K(x, t)$ and given function $g(x)$, $\lambda$ is a real number. Numerically, Fredholm equations of the first kind are very ill-conditioned. We will consider equations of the second kind only.

The basic idea is to approximate the integral by some quadrature formula

$$
\int_a^b \! \xi(t)\, dt \approx \sum_j^N w_j\, \xi(t_j)
$$

with appropriate weights $w_j$ and nodes $t_j$ and $j=1, \dots, N$. The accuracy of the approximation is controlled by $N$.

This way, the FE is approximated by 

$$
\lambda \sum_j w_j\, K(x, t_j)\, \color{blue}{f(t_j)} + g(x) = \color{blue}{f(x)}
$$

Note that here $x$ is a continuous variable, and we only discretized $t$.


Evaluating this equation on the grid $x = t_k$, we obtain

$$
\lambda \sum_j w_j \, K_{k, j}\, \color{blue}{f_j} + g_k = \color{blue}{f_k}
$$

where $f_j \equiv f(t_j)$, $g_j \equiv g(t_j)$ and $K_{k, j} \equiv K(t_k, t_j)$. This is nothing but a system of linear algebraic equations for the vector of $\color{blue}{f_j}$.

Its solution gives the values of the unknown function $f(x)$ at the discrete values $x=t_j$ for $j=1, \dots, N$.

### Task 2.1
(20% of the grade)

Solve an example Fredholm equation of the second kind

$$
f(x) = \sin(\pi x)  + \frac{1}{2} \int_0^1\! f(t) \, dt
$$

Here $a, b = 0, 1$, $\lambda = \dfrac{1}{2}$, the kernel is $K(x, t) = 1$ and the right-hand side $g(x) = \sin(\pi x)$.

In fact, the exact solution is (В. А. Попов, Сборник задач по интегральным уравнениям, 2006, стр. 5)

$$
f(x) = \sin(\pi x) + \frac{2}{\pi}
$$

For the integral, we can use a Gaussian quadrature with the weight function $w(x) = 1$ on $(0, 1)$. Looking at http://dlmf.nist.gov/18.3, and find the corresponding function in *scipy.special* (pay attention to the integration interval).

Compare the obtained function values with the true solution for different values of $N$. What value of $N$ can you stop at?

In [0]:
### Enter your code here

### Task 2.2
(20% of the grade)

An obvious issue with the privious result for $f(x)$ that it returns the solution on a fixed set of points. We do not control precise positions of these points.

1. First, let's try to interpolate the values obtained in the previous task to a uniform grid of length 50. You may use any interpolation method. Compare the interpolation values with exact solution values.

In [0]:
### Enter your code here

2. Now use the following formula to calculate the function at points of the same uniform grid.
$$
f(x) = g(x) + \lambda \sum_j^N w_j\, K(x, t_j)\, f(t_j)
$$
Again, compare the interpolation values with the exact solution values, and also with the direct interpolation method.

In [0]:
### Enter your code here